In [ ]:
# Installation of required libraries
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

In [ ]:
# load the dataset as pandas dataframe
# Note: Please update the path to your local file path for this to work
df = pd.read_csv(r" ")

In [ ]:
# Drop last 2 columns of dataframe
#df = df.iloc[: , :-2]
df.drop(columns=df.columns[-2:], axis=1, inplace=True)
df.head(5)

In [ ]:
# replacing categorical column to numarical columns
df['Attrition_Flag'].replace(['Existing Customer', 'Attrited Customer'],[1, 0], inplace=True)

In [ ]:
# replacing categorical column to numarical columns using OneHotEncoder

from sklearn.preprocessing import OneHotEncoder

final_df = df
categorical_columns = ['Gender', 'Education_Level', 'Marital_Status','Income_Category', 'Card_Category']
for col in categorical_columns:
    col_df = pd.get_dummies(df[col], prefix=col)
    final_df = pd.concat((final_df, col_df), axis=1).drop(col, axis=1)

In [ ]:
# select the features to use in models 
X = final_df[[ 'Credit_Limit','Total_Revolving_Bal','Months_Inactive_12_mon', 'Total_Trans_Amt' , 
              'Avg_Utilization_Ratio']] 

In [ ]:
# Elbow method to find optimal number of clusters
from sklearn.pipeline import Pipeline
wcss=[]
for i in range(1,11):
    pipe_elbow = Pipeline([
        ('scaler', StandardScaler()),
        ('pca', PCA(2)),
        ('Kmeans', KMeans(n_clusters=i))
    ])
    pipe_elbow.fit(X)
    wcss.append(pipe_elbow[-1].inertia_)

plt.plot(range(1,11),wcss)

In [ ]:
# pipe line to preform scaler, pca and Kmeans modeling in single step
pipe_model = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(2)),
    ('Kmeans', KMeans(n_clusters=3))])
y_means=pipe_model.fit_predict(X)

In [ ]:
# ploting the clusters
X_pca = pipe_model[:-1].transform(X)
plt.scatter(X_pca[:,0],X_pca[:,1],c=y_means)

In [ ]:
# calculating score of model
from sklearn.metrics import silhouette_score
score = silhouette_score(X_pca,pipe_model[-1].labels_,metric='euclidean')
score

In [ ]:
# save the pipeline to a file
import joblib
joblib.dump(pipe_model, 'pipe_model.joblib')